In [ ]:
# Установка необходимых библиотек
!pip install ultralytics
!pip install torch torchvision torchaudio

import os
import zipfile
import pandas as pd
import matplotlib.pyplot as plt
from ultralytics import YOLO
from PIL import Image
import cv2

# Подключение Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Пути к файлам на Google Drive
main_path = "/content/drive/MyDrive"
zip_path = os.path.join(main_path, "signs.zip")
extracted_path = "/content/signs"

# Распаковка архива
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_path)

# Проверка наличия файлов после распаковки
print("Содержимое директории после распаковки:", os.listdir(extracted_path))

# Пути к папкам и файлам после распаковки
csv_train_path = os.path.join(extracted_path, 'rtsd-d3-gt/blue_rect/train_gt.csv')
csv_test_path = os.path.join(extracted_path, 'rtsd-d3-gt/blue_rect/test_gt.csv')
images_train_path = os.path.join(extracted_path, 'rtsd-d3-frames/train')
images_test_path = os.path.join(extracted_path, 'rtsd-d3-frames/test')
output_train_path = os.path.join(extracted_path, 'Results/train')
output_test_path = os.path.join(extracted_path, 'Results/test')

# Создание выходных директорий для YOLO-меток, если они не существуют
os.makedirs(output_train_path, exist_ok=True)
os.makedirs(output_test_path, exist_ok=True)

def convert_to_yolo_format(csv_path, images_path, output_path):
    if not os.path.exists(csv_path):
        print(f"CSV файл {csv_path} не найден!")
        return
    data = pd.read_csv(csv_path)
    for _, row in data.iterrows():
        filename = row['filename']
        x_from, y_from, width, height = row['x_from'], row['y_from'], row['width'], row['height']
        class_id = row['sign_class']
        
        image_path = os.path.join(images_path, filename)
        
        # Проверка наличия изображения
        if not os.path.exists(image_path):
            print(f"Image {image_path} not found, skipping.")
            continue
        
        image = plt.imread(image_path)
        img_height, img_width = image.shape[:2]

        x_center = (x_from + width / 2) / img_width
        y_center = (y_from + height / 2) / img_height
        w = width / img_width
        h = height / img_height

        label_path = os.path.join(output_path, os.path.splitext(filename)[0] + '.txt')
        with open(label_path, 'a') as f:
            f.write(f"{class_id} {x_center} {y_center} {w} {h}\n")

# Проверка наличия файлов CSV перед конвертацией
if os.path.exists(csv_train_path):
    convert_to_yolo_format(csv_train_path, images_train_path, output_train_path)
else:
    print(f"CSV файл {csv_train_path} не найден!")

if os.path.exists(csv_test_path):
    convert_to_yolo_format(csv_test_path, images_test_path, output_test_path)
else:
    print(f"CSV файл {csv_test_path} не найден!")

# Проверка наличия изображений после конвертации
print("Файлы в директории train после конвертации:", os.listdir(output_train_path))
print("Файлы в директории test после конвертации:", os.listdir(output_test_path))

# Подготовка конфигурационного файла и файла данных
config_path = 'yolov8s.yaml'
data_path = 'data.yaml'

# Пример конфигурационного файла yolov8s.yaml
config_content = """
# YOLOv8s Configuration

# Model parameters
network: yolov8s
depth_multiple: 0.33
width_multiple: 0.50

# Train parameters
train:
  img_size: 640  # Размер входного изображения
  epochs: 300  # Количество эпох
  batch: 16  # Размер batch
  optimizer: Adam  # Оптимизатор
  learning_rate: 0.001  # Скорость обучения
  momentum: 0.937  # Момент
  weight_decay: 0.0005  # Регуляризация веса
  warmup_epochs: 3.0  # Эпохи прогрева
  warmup_momentum: 0.8  # Момент прогрева
  warmup_bias_lr: 0.1  # Скорость обучения смещения прогрева

# Augmentation parameters
augmentation:
  mosaic: 1.0  # mosaic augmentation probability
  hsv_h: 0.015  # цветовые искажения: оттенок
  hsv_s: 0.7  # цветовые искажения: насыщенность
  hsv_v: 0.4  # цветовые искажения: значение
  degrees: 0.0  # угол поворота
  translate: 0.1  # перевод
  scale: 0.5  # масштабирование
  shear: 0.0  # сдвиг
  flipud: 0.0  # Вертикальное отображение
  fliplr: 0.5  # Горизонтальное отображение
  mosaic_border: [-320, -320]  # граница мозайки

# Anchors
anchors:
  - [10, 13, 16, 30, 33, 23]
  - [30, 61, 62, 45, 59, 119]
  - [116, 90, 156, 198, 373, 326]

# Dataset parameters
data:
  train: /content/signs/Results/train  # Путь к тренировочному датасету
  val: /content/signs/Results/test  # Путь к валидационному датасету

# Number of classes
nc: 10  # Укажите точное количество классов знаков

# Class names
names:
  0: 'stop'
  1: 'pedestrian_crossing'
  2: 'yield'
  3: 'speed_limit_50'
  4: 'no_entry'
  5: 'one_way'
  6: 'no_parking'
  7: 'bicycle_crossing'
  8: 'school_zone'
  9: 'main_road'
"""
with open(config_path, 'w') as f:
    f.write(config_content)

# Пример файла данных data.yaml
data_content = """
train: /content/signs/Results/train
val: /content/signs/Results/test
nc: 10  # Number of classes (здесь укажите точное количество классов знаков)
names:
  0: 'stop'
  1: 'pedestrian_crossing'
  2: 'yield'
  3: 'speed_limit_50'
  4: 'no_entry'
  5: 'one_way'
  6: 'no_parking'
  7: 'bicycle_crossing'
  8: 'school_zone'
  9: 'main_road'
"""
with open(data_path, 'w') as f:
    f.write(data_content)

# Проверка доступности GPU
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Используемое устройство:", device)

# Создание модели YOLOv8s с нуля и обучение модели
model = YOLO('yolov8s.yaml').to(device)

# Проверка наличия изображений перед обучением
if not os.listdir(output_train_path):
    print("Нет изображений в директории train!")
else:
    print("Изображения найдены в директории train!")

if not os.listdir(output_test_path):
    print("Нет изображений в директории test!")
else:
    print("Изображения найдены в директории test!")

# Обучение модели, если изображения найдены
if os.listdir(output_train_path) and os.listdir(output_test_path):
    model.train(data=data_path, epochs=20, imgsz=640, batch=16, project='runs/train')
else:
    print("Обучение не может быть начато, так как изображения не найдены в одной или обеих директориях.")

# Визуализация на случайном изображении из директории test
test_images = os.listdir(output_test_path)
if test_images:
    image_path = os.path.join(output_test_path, test_images[0]) # Берем первое изображение из директории test
    if os.path.exists(image_path):
        print(f"Визуализация изображения: {image_path}")
        results = model(image_path)
        results.show()
    else:
        print(f"Изображение {image_path} не найдено!")
else:
    print("Нет изображений в директории test для визуализации.")

# Визуализация на видео
video_day = os.path.join(main_path, "Брест_день.mp4")
video_night = os.path.join(main_path, "Брест_ночь.mp4")
output_dir = "/content/drive/MyDrive/ColabNotebooks/lab3-Oivis/out"

model.predict(source=video_day, save=True, save_dir=output_dir)
model.predict(source=video_night, save=True, save_dir=output_dir)
